<a href="https://colab.research.google.com/github/yaminsu5674/Cap_Hipsterizer/blob/main/Cat_lmks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Because of GPU environment I trained model_lmks on colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import torch
import datetime
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from torchsummary import summary

import random
import numpy as np
import datetime

CUDA_ABAILABLE=torch.cuda.is_available()
device=torch.device('cuda' if CUDA_ABAILABLE else 'cpu')
print('device : {}'.format(device))

random.seed(23)
torch.manual_seed(23)
if device=='cuda':
  torch.cuda.manual_seed_all(23)



device : cuda


In [ ]:
img_size=224

mode='lmks'
if mode is 'bbs':
  output_size=4
elif mode is 'lmks':
  output_size=18

start_time=datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

data_00=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_00.npy',allow_pickle=True)
data_01=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_01.npy',allow_pickle=True)
data_02=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_02.npy',allow_pickle=True)
data_03=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_03.npy',allow_pickle=True)
data_04=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_04.npy',allow_pickle=True)
data_05=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_05.npy',allow_pickle=True)
data_06=np.load('/content/gdrive/MyDrive/Downloads/Cat_hipsterizer/train_lmks_data/lmks_CAT_06.npy',allow_pickle=True)

x_train = np.concatenate((data_00.item().get('imgs'), data_01.item().get('imgs'), data_02.item().get('imgs'), data_03.item().get('imgs'), data_04.item().get('imgs'), data_05.item().get('imgs')), axis=0)
y_train = np.concatenate((data_00.item().get(mode), data_01.item().get(mode), data_02.item().get(mode), data_03.item().get(mode), data_04.item().get(mode), data_05.item().get(mode)), axis=0)

x_test=np.array(data_06.item().get('imgs'))
y_test=np.array(data_06.item().get(mode))

x_train=x_train.astype('float32')/255.
x_test=x_test.astype('float32')/255.
x_train=np.reshape(x_train,(-1,3,img_size,img_size))
x_test=np.reshape(x_test,(-1,3,img_size,img_size))

y_train=np.reshape(y_train,(-1,output_size))
y_test=np.reshape(y_test,(-1,output_size))

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,x_train,y_train):
        self.x_data=x_train
        self.y_data=y_train

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        x=torch.FloatTensor(self.x_data[idx])
        y=torch.FloatTensor(self.y_data[idx])
        return x,y

dataset=CustomDataset(x_train,y_train)

data_loader=DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
)

In [ ]:
model_lmks=torchvision.models.mobilenet_v2(pretrained=True)
model=torchvision.models.mobilenet_v2()
#for param in model_lmks.parameters():
#  param.requires_grad=False
num_ftrs=model_lmks.classifier[-1].in_features
model_lmks.classifier=nn.Sequential(
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            #nn.Linear(640,128),
            #nn.ReLU6(inplace=True),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,output_size)   
)

'''model_lmks.classifier[-1]=nn.Linear(num_ftrs,128)
model_lmks.classifier.add_module('plus',nn.Sequential(
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,output_size)
))'''

model_lmks=model_lmks.to(device)
criterion=nn.MSELoss().to(device)
optimizer=optim.Adam(model_lmks.parameters())

In [ ]:
summary(model_lmks,(3,img_size,img_size))

In [ ]:
#model_lmks1=torch.load('/content/gdrive/MyDrive/Trained_Models/'+'model_lmks_98')

In [ ]:
#summary(model_lmks1,(3,img_size,img_size))

In [ ]:
training_epochs=250

for t in range(2):
  status=0
  for epoch in range(training_epochs+1):
    avg_cost=0

    for X,Y in data_loader:
      X=X.to(device)
      Y=Y.to(device)

      pred=model_lmks(X)
      optimizer.zero_grad()
      cost=criterion(pred,Y)
      cost.backward()
      optimizer.step()
      avg_cost+=cost

    avg_cost=avg_cost/32
    status=avg_cost
    if epoch%25==0 and epoch!=0:
      print('Training Epochs: {}, Avg_Cost : {}'.format(epoch,avg_cost))
  time=datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
  torch.save(model_lmks.state_dict(),'/content/gdrive/MyDrive/Trained_Models/'+time+'model_lmks.pt')
  print('------------------ {} time ended!---------------------')
  